In [1]:
from utils.molec_class_files import r14, r32, r50, r125, r134a, r143a, r170, r41, r23, r161, r152a, r152, r134, r143, r116
from utils import atom_type, opt_atom_types
import numpy as np
import unyt as u
import pandas as pd
from fffit.fffit.utils import values_real_to_scaled, values_scaled_to_real, variances_scaled_to_real, generate_lhs
from fffit.fffit.plot import plot_obj_contour
import os

#Load class properies for each molecule
r14_class = r14.R14Constants()
r32_class = r32.R32Constants()
r50_class = r50.R50Constants()
r125_class = r125.R125Constants()
r134a_class = r134a.R134aConstants()
r143a_class = r143a.R143aConstants()
r170_class = r170.R170Constants()

r41_class = r41.R41Constants()
r23_class = r23.R23Constants()
r161_class = r161.R161Constants()
r152a_class = r152a.R152aConstants()
r152_class = r152.R152Constants()
r143_class = r143.R143Constants()
r134_class = r134.R134Constants()
r116_class = r116.R116Constants()

2024-07-19 14:34:42.515074: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-19 14:34:42.561204: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-19 14:34:42.561260: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-19 14:34:42.562742: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-19 14:34:42.570112: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-19 14:34:42.570586: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
values = np.array([1,2,3,4,5])
print(values[:1])
sets = np.array([[0,0,0],
                [1,1,1],
                [1,1,1],
                [1,0.5,1]])

[1]


In [3]:
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import pdist, squareform
def filter_unique_sets(sets, threshold=0.01):
    # Compute the pairwise Euclidean distances
    distances = pdist(sets, metric='euclidean')
    distance_matrix = squareform(distances)

    # Initialize a boolean array to keep track of unique sets
    unique_mask = np.ones(sets.shape[0], dtype=bool)

    # Iterate over the upper triangle of the distance matrix
    for i in range(sets.shape[0]):
        # If the current set is already marked as non-unique, skip it
        if not unique_mask[i]:
            continue
        # Mark sets within the threshold distance as non-unique
        within_threshold = distance_matrix[i] <= threshold
        unique_mask[within_threshold] = False
        unique_mask[i] = True  # Keep the current set

    return sets[unique_mask]

# Example usage:
sets = np.array([
    [0, 0, 0],
    [1, 1, 1],
    [1, 1, 1],
    [1, 0.5, 1]])

print(filter_unique_sets(sets))

def get_unique_sets(self, all_param_sets, save_data = False, save_label = None):
        """
        Gets unique sets of parameters from an array of parameters
        
        """
        assert isinstance(save_data, bool), "save_data must be a bool"
        assert isinstance(save_label, (str, type(None))), "save_label must be a string or None"
        #Scale values from preferred to real units
        all_param_sets_real = self.values_pref_to_real(all_param_sets)
        #Scale values between 0 and 1 with minmax scaler
        scaler = MinMaxScaler()
        scaler.fit(self.at_class.at_bounds_nm_kjmol.T)
        all_param_sets_scaled = scaler.transform(all_param_sets_real)
        #Calculate the scaled euclidean distance between each pair of scaled points
        dist = pdist(all_param_sets_scaled)/np.sqrt(all_param_sets.shape[1])
        #Convert the condensed distance matrix to square form
        dist_sq = squareform(dist)
        #Fill diagonals w/ infinity
        np.fill_diagonal(dist_sq, np.inf)
        # Find the indices of points where all distances to other points (excluding diagonal) are greater than 0.01
        valid_indices = np.where(np.all(dist_sq > 0.01, axis=1))[0]
        if len(valid_indices) > 0:
            unique_param_sets = all_param_sets[valid_indices]
        else:
            unique_param_sets = all_param_sets[0].reshape(1,-1)

        data_df = pd.DataFrame(unique_param_sets, columns=self.at_class.at_names)

        if save_data == True:
            save_label = save_label if save_label is not None else "test_set"
            save_csv_path = os.path.join(self.use_dir_name, "unique_" + save_label + ".csv")
            data_df.to_csv(save_csv_path, index = False, header = True)

        return data_df

     0    1    2
0  0.0  0.0  0.0
1  1.0  0.5  1.0


In [9]:
#Get obj from a set of parameters
import copy
# at_class = atom_type.AT_Scheme_11()
save_data = True
obj_choice = "ExpVal"
molec_names = ["R14", "R32", "R50", "R170", "R125", "R134a", "R143a"] 
at_number = 11
problem_setup = opt_atom_types.Problem_Setup(molec_names, at_number, obj_choice)

#Best set from Experiment
#Set parameter set of interest (in this case get the best parameter set)
all_molec_dir = problem_setup.use_dir_name
path_best_sets = os.path.join(all_molec_dir, "best_per_run.csv")
assert os.path.exists(path_best_sets), "best_per_run.csv not found in directory"
all_df = pd.read_csv(path_best_sets, header = 0)
first_param_name = problem_setup.at_class.at_names[0] + "_min"
last_param_name = problem_setup.at_class.at_names[-1] + "_min"
best_set = all_df.loc[:3, first_param_name:last_param_name].values
best_real = problem_setup.values_pref_to_real(copy.copy(best_set))
best_pref = problem_setup.values_real_to_pref(copy.copy(best_real))
print(best_pref)
 #Unscale data from 0 to 1 to get correct objective values
at_bounds_pref = problem_setup.at_class.at_bounds_nm_kjmol
theta_guess = values_real_to_scaled(best_real.reshape(-1,22), at_bounds_pref)

[[ 3.8239609   3.39920875  3.8546543   3.33491516  3.89283056  3.639822
   2.0802739   3.27818297  2.82286278  2.94406375  2.92954002 55.18038398
  42.25299831 44.52003378 64.60461016 23.16107594 38.57869402 10.34133395
  31.32968437 42.44898523 22.10720589 28.52377367]
 [ 3.82754911  3.39798447  3.85549928  3.33258007  3.89432735  3.63465496
   2.07673867  3.29733467  2.82270312  2.94353075  2.92961987 55.4788963
  42.30334122 44.65694564 65.05272146 23.15428879 38.5826963  10.30499307
  31.32274285 42.44767029 22.19684474 28.52768862]
 [ 3.82533802  3.39763371  3.85552511  3.33361125  3.89140049  3.63812591
   2.07605843  3.28491134  2.82278166  2.94461944  2.92961392 55.41485728
  42.29773532 44.65670806 65.11913255 23.2843161  38.43964866 10.32362972
  31.07092867 42.42104229 22.17778174 28.53608863]
 [ 3.82577359  3.39794495  3.85452641  3.3407888   3.88913186  3.63039545
   2.07414236  3.26305209  2.82301276  2.94730451  2.92963941 55.46630129
  42.30975767 44.67950372 64.0910677

In [ ]:
repeats = 1
seed =1
setup = opt_atom_types.Opt_ATs(molec_data_dict, all_gp_dict, at_class, repeats, seed, obj_choice)
rank_parameters, n_data = setup.rank_parameters(best_real)

In [ ]:
#Print the parameter name asociated with each index of rank_parameters
rank_parameters_names = [problem_setup.at_class.at_names[i] for i in rank_parameters]
print(rank_parameters_names)
print(n_data)

In [ ]:
opt_num_params, rcc, loss_k, loss_k_params = setup.estimate_opt(theta_guess, rank_parameters, n_data)
print("Optimal number of parameters: ", opt_num_params)
print("RCC: ", rcc)
print("Best Param Set: ", loss_k_params[opt_num_params-1])

In [ ]:
import numpy as np
import numdifftools as nd
fun = lambda x: problem_setup.calc_fxn(x)
dfun = nd.Gradient(fun)
gradient = dfun([theta_guess])

In [ ]:
gradient_scl = gradient*theta_guess
print(gradient_scl.shape)

In [ ]:
import numpy as np
from numpy.linalg import norm
from scipy.linalg import lstsq

def rank_parameters(Z):
    # Step 1: Calculate the magnitude of each column in Z
    column_magnitudes = norm(Z, axis=0)
    
    # Initialize variables
    ranked_indices = []  # To keep track of ranked parameter indices
    k = 1
    n, m = Z.shape

    while k <= m:
        if k == 1:
            # Step 2: Identify the most estimable parameter
            max_index = np.argmax(column_magnitudes)
            ranked_indices.append(max_index)
        else:
            # Step 3: Build X_k with the k most estimable columns
            X_k = Z[:, ranked_indices]

            # Predict Z using ordinary least-squares
            Z_hat, _, _, _ = lstsq(X_k, Z)

            # Calculate the residual matrix R_k
            R_k = Z - X_k @ Z_hat

            # Step 4: Calculate the magnitude of each column in R_k
            residual_magnitudes = norm(R_k, axis=0)

            # Step 5: Determine the next most estimable parameter
            # Ensure we pick a column that hasn't been ranked yet
            for idx in np.argsort(-residual_magnitudes):
                if idx not in ranked_indices:
                    ranked_indices.append(idx)
                    break
        
        k += 1  # Step 6: Increase k and repeat

    return ranked_indices

# Example usage
Z = gradient_scl
ranked_params = rank_parameters(Z)
print("Ranked parameters (indices):", ranked_params)
value = [at_class.at_names[param] for param in ranked_params]
print(value)